In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
import math
import copy
import random
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
from hybrid_network import *
from hybrid_util import *
learning_rate = 0.001

In [2]:
trainloader, testloader = get_cifar()                                
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = Cifar_Small_ConvNet(
    non_negative = [False, False, False, False],
    norm = [False, False, False, False]
)
net = net.to(device)

Files already downloaded and verified
Files already downloaded and verified
cuda:0


In [3]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200000,250000, 300000], gamma=0.1)
eps = 2/255 * 1.1
running_eps = 0
epoch = 0
itr = 0
k = 0
while itr < 350000:
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        net.train() 
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        loss = 0
        optimizer.zero_grad()
        
        outputs = net(torch.cat([inputs, inputs], 0))
        outputs  = outputs[:outputs.shape[0]//2]
        loss += (1 - k) * criterion(outputs, labels)
        
        if itr > 10000 and itr < 160000:
            running_eps += eps/150000
            k += 0.5/150000
   
        if itr > 10000:
            x_ub = inputs + running_eps
            x_lb = inputs - running_eps
            outputs = net.forward(torch.cat([x_ub, x_lb], 0))
            z_hb = outputs[:outputs.shape[0]//2]
            z_lb = outputs[outputs.shape[0]//2:]
            lb_mask = torch.eye(10).cuda()[labels]
            hb_mask = 1 - lb_mask
            outputs = z_lb * lb_mask + z_hb * hb_mask
            loss += k * criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        itr+=1
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 250))
    net.eval()
    print_accuracy(net, trainloader, testloader, device, test=True, eps = 0)
    print_accuracy(net, trainloader, testloader, device, test=True, eps = running_eps)
    if itr > 250000:
        print_accuracy(net, trainloader, testloader, device, test=True, eps = 2/255)
    epoch+= 1

[1,   250] loss: 1.677
Accuracy of the network on the 10000 test images:  0.4852
Accuracy of the network on the 10000 test images:  0.4811
[2,   250] loss: 1.404
Accuracy of the network on the 10000 test images:  0.5272
Accuracy of the network on the 10000 test images:  0.5285
[3,   250] loss: 1.294
Accuracy of the network on the 10000 test images:  0.5517
Accuracy of the network on the 10000 test images:  0.5561
[4,   250] loss: 1.238
Accuracy of the network on the 10000 test images:  0.5727
Accuracy of the network on the 10000 test images:  0.5698
[5,   250] loss: 1.177
Accuracy of the network on the 10000 test images:  0.5851
Accuracy of the network on the 10000 test images:  0.5842
[6,   250] loss: 1.145
Accuracy of the network on the 10000 test images:  0.5933
Accuracy of the network on the 10000 test images:  0.5939
[7,   250] loss: 1.113
Accuracy of the network on the 10000 test images:  0.5928
Accuracy of the network on the 10000 test images:  0.5969
[8,   250] loss: 1.082
Accu

[60,   250] loss: 0.891
Accuracy of the network on the 10000 test images:  0.6937
Accuracy of the network on the 10000 test images:  0.1502
[61,   250] loss: 0.894
Accuracy of the network on the 10000 test images:  0.6934
Accuracy of the network on the 10000 test images:  0.1653
[62,   250] loss: 0.893
Accuracy of the network on the 10000 test images:  0.6911
Accuracy of the network on the 10000 test images:  0.1572
[63,   250] loss: 0.891
Accuracy of the network on the 10000 test images:  0.6931
Accuracy of the network on the 10000 test images:  0.1562
[64,   250] loss: 0.897
Accuracy of the network on the 10000 test images:  0.6948
Accuracy of the network on the 10000 test images:  0.176
[65,   250] loss: 0.898
Accuracy of the network on the 10000 test images:  0.6981
Accuracy of the network on the 10000 test images:  0.1864
[66,   250] loss: 0.896
Accuracy of the network on the 10000 test images:  0.6963
Accuracy of the network on the 10000 test images:  0.1808
[67,   250] loss: 0.8

Accuracy of the network on the 10000 test images:  0.3023
[119,   250] loss: 0.963
Accuracy of the network on the 10000 test images:  0.6873
Accuracy of the network on the 10000 test images:  0.2897
[120,   250] loss: 0.966
Accuracy of the network on the 10000 test images:  0.689
Accuracy of the network on the 10000 test images:  0.2881
[121,   250] loss: 0.970
Accuracy of the network on the 10000 test images:  0.6818
Accuracy of the network on the 10000 test images:  0.2932
[122,   250] loss: 0.968
Accuracy of the network on the 10000 test images:  0.6823
Accuracy of the network on the 10000 test images:  0.2902
[123,   250] loss: 0.969
Accuracy of the network on the 10000 test images:  0.6828
Accuracy of the network on the 10000 test images:  0.2905
[124,   250] loss: 0.974
Accuracy of the network on the 10000 test images:  0.6857
Accuracy of the network on the 10000 test images:  0.2903
[125,   250] loss: 0.973
Accuracy of the network on the 10000 test images:  0.684
Accuracy of the

[177,   250] loss: 1.046
Accuracy of the network on the 10000 test images:  0.6708
Accuracy of the network on the 10000 test images:  0.3376
[178,   250] loss: 1.045
Accuracy of the network on the 10000 test images:  0.6691
Accuracy of the network on the 10000 test images:  0.331
[179,   250] loss: 1.044
Accuracy of the network on the 10000 test images:  0.6735
Accuracy of the network on the 10000 test images:  0.3341
[180,   250] loss: 1.045
Accuracy of the network on the 10000 test images:  0.6693
Accuracy of the network on the 10000 test images:  0.3351
[181,   250] loss: 1.053
Accuracy of the network on the 10000 test images:  0.6681
Accuracy of the network on the 10000 test images:  0.3204
[182,   250] loss: 1.052
Accuracy of the network on the 10000 test images:  0.6719
Accuracy of the network on the 10000 test images:  0.3282
[183,   250] loss: 1.051
Accuracy of the network on the 10000 test images:  0.6738
Accuracy of the network on the 10000 test images:  0.3238
[184,   250] l

Accuracy of the network on the 10000 test images:  0.6608
Accuracy of the network on the 10000 test images:  0.3435
[236,   250] loss: 1.122
Accuracy of the network on the 10000 test images:  0.6618
Accuracy of the network on the 10000 test images:  0.3339
[237,   250] loss: 1.126
Accuracy of the network on the 10000 test images:  0.6579
Accuracy of the network on the 10000 test images:  0.3352
[238,   250] loss: 1.123
Accuracy of the network on the 10000 test images:  0.6547
Accuracy of the network on the 10000 test images:  0.3356
[239,   250] loss: 1.131
Accuracy of the network on the 10000 test images:  0.6512
Accuracy of the network on the 10000 test images:  0.3417
[240,   250] loss: 1.130
Accuracy of the network on the 10000 test images:  0.6573
Accuracy of the network on the 10000 test images:  0.3469
[241,   250] loss: 1.130
Accuracy of the network on the 10000 test images:  0.6551
Accuracy of the network on the 10000 test images:  0.3345
[242,   250] loss: 1.133
Accuracy of t

...

[1111,   250] loss: 1.535
Accuracy of the network on the 10000 test images:  0.5556
Accuracy of the network on the 10000 test images:  0.3482
[1112,   250] loss: 1.536
Accuracy of the network on the 10000 test images:  0.5547
Accuracy of the network on the 10000 test images:  0.349
[1113,   250] loss: 1.538
Accuracy of the network on the 10000 test images:  0.5469
Accuracy of the network on the 10000 test images:  0.3415
[1114,   250] loss: 1.537
Accuracy of the network on the 10000 test images:  0.5523
Accuracy of the network on the 10000 test images:  0.3474
[1115,   250] loss: 1.538
Accuracy of the network on the 10000 test images:  0.5516
Accuracy of the network on the 10000 test images:  0.3474
[1116,   250] loss: 1.537
Accuracy of the network on the 10000 test images:  0.5541
Accuracy of the network on the 10000 test images:  0.348
[1117,   250] loss: 1.536
Accuracy of the network on the 10000 test images:  0.5612
Accuracy of the network on the 10000 test images:  0.3381
[1118,  

[1169,   250] loss: 1.538
Accuracy of the network on the 10000 test images:  0.5569
Accuracy of the network on the 10000 test images:  0.3458
[1170,   250] loss: 1.537
Accuracy of the network on the 10000 test images:  0.555
Accuracy of the network on the 10000 test images:  0.3421
[1171,   250] loss: 1.533
Accuracy of the network on the 10000 test images:  0.5581
Accuracy of the network on the 10000 test images:  0.3437
[1172,   250] loss: 1.535
Accuracy of the network on the 10000 test images:  0.5521
Accuracy of the network on the 10000 test images:  0.3451
[1173,   250] loss: 1.537
Accuracy of the network on the 10000 test images:  0.5582
Accuracy of the network on the 10000 test images:  0.337
[1174,   250] loss: 1.533
Accuracy of the network on the 10000 test images:  0.558
Accuracy of the network on the 10000 test images:  0.3432
[1175,   250] loss: 1.537
Accuracy of the network on the 10000 test images:  0.553
Accuracy of the network on the 10000 test images:  0.3521
[1176,   2

[1227,   250] loss: 1.531
Accuracy of the network on the 10000 test images:  0.5575
Accuracy of the network on the 10000 test images:  0.3473
[1228,   250] loss: 1.539
Accuracy of the network on the 10000 test images:  0.5503
Accuracy of the network on the 10000 test images:  0.3577
[1229,   250] loss: 1.536
Accuracy of the network on the 10000 test images:  0.5543
Accuracy of the network on the 10000 test images:  0.3496
[1230,   250] loss: 1.530
Accuracy of the network on the 10000 test images:  0.55
Accuracy of the network on the 10000 test images:  0.3398
[1231,   250] loss: 1.535
Accuracy of the network on the 10000 test images:  0.5576
Accuracy of the network on the 10000 test images:  0.3455
[1232,   250] loss: 1.538
Accuracy of the network on the 10000 test images:  0.5521
Accuracy of the network on the 10000 test images:  0.3433
[1233,   250] loss: 1.531
Accuracy of the network on the 10000 test images:  0.5616
Accuracy of the network on the 10000 test images:  0.3512
[1234,  

[1285,   250] loss: 1.534
Accuracy of the network on the 10000 test images:  0.5543
Accuracy of the network on the 10000 test images:  0.3481
[1286,   250] loss: 1.533
Accuracy of the network on the 10000 test images:  0.5499
Accuracy of the network on the 10000 test images:  0.3502
[1287,   250] loss: 1.533
Accuracy of the network on the 10000 test images:  0.5555
Accuracy of the network on the 10000 test images:  0.353
[1288,   250] loss: 1.534
Accuracy of the network on the 10000 test images:  0.5598
Accuracy of the network on the 10000 test images:  0.3425
[1289,   250] loss: 1.532
Accuracy of the network on the 10000 test images:  0.5571
Accuracy of the network on the 10000 test images:  0.3468
[1290,   250] loss: 1.536
Accuracy of the network on the 10000 test images:  0.5567
Accuracy of the network on the 10000 test images:  0.3438
[1291,   250] loss: 1.531
Accuracy of the network on the 10000 test images:  0.5548
Accuracy of the network on the 10000 test images:  0.3485
[1292, 

[1343,   250] loss: 1.535
Accuracy of the network on the 10000 test images:  0.5577
Accuracy of the network on the 10000 test images:  0.3434
[1344,   250] loss: 1.533
Accuracy of the network on the 10000 test images:  0.5457
Accuracy of the network on the 10000 test images:  0.3484
[1345,   250] loss: 1.532
Accuracy of the network on the 10000 test images:  0.5516
Accuracy of the network on the 10000 test images:  0.3522
[1346,   250] loss: 1.532
Accuracy of the network on the 10000 test images:  0.554
Accuracy of the network on the 10000 test images:  0.3511
[1347,   250] loss: 1.534
Accuracy of the network on the 10000 test images:  0.5602
Accuracy of the network on the 10000 test images:  0.3478
[1348,   250] loss: 1.534
Accuracy of the network on the 10000 test images:  0.5548
Accuracy of the network on the 10000 test images:  0.3428
[1349,   250] loss: 1.535
Accuracy of the network on the 10000 test images:  0.5592
Accuracy of the network on the 10000 test images:  0.3524
[1350, 

In [6]:
print_accuracy(net, trainloader, testloader, device, test=True, eps = 2/255)

Accuracy of the network on the 10000 test images:  0.3615


0.3615